In [47]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.integrate import quad,dblquad,nquad
from scipy.integrate import solve_ivp

# Volume based population balance model
## Continuous volume based breakage equation
Let $v(l,t)$ be the volume density function with size $l$ at time $t$. Then  $v(l,t)dl$ be the volume fraction of particles with sizes from $l$ to $l+dl$ at time $t$ and $\int_0^\infty v(l,t)dl=1$. The continuous volume based breakage equation is
\begin{equation}
\frac{\partial v(l,t)}{\partial t}=\int_{l}^\infty S(x)b(l,x)v(x,t)dx-S(l)v(l,t)
\end{equation}
where $b(l,x)$ is the breakage function and $S(x)$ is the selection rate function. The breakage function is the probabiltiy of particles with size $x$ broken into the size range $l$ to $l+dl$. The selection function is the specific rate of breakage which is volume fraction of particles of size $l$ broken per unit time. The volume density function and breakage functions are dimensionless and the dimension of selection function is $[\text{T}^{-1}]$.

The mass of particles with sizes less that $l$ is given by
\begin{equation}
M(l,t)=\int_0^lm(x,t)dx
\end{equation}
and the $k$th moment is defined as
\begin{equation}
M_k(t)=\int_0^\infty x^km(x,t)dx
\end{equation}

## Discretized breakage equation
Let interval $i$ denotes the particle sizes from $l_{i-1}$ to $l_i$ and $V_i(t)$ denotes the volume fraction of particles in interval $i$ at time t. Then
\begin{equation}
V_i(t)=\int_{l_{i-1}}^{l_i}v(l,t)dx
\end{equation}
## Discretized breakage birth
The rate of breakage birth is
\begin{equation}
R_i^{[1]}=\sum_{j=i}^{n}b_{ij}S_jV_j
\end{equation}
where $S_i$ is the discretized selection function for interval $i$ and $b_{ij}$ is discretized breakage function which is the volume fraction of fragments broken from interval $j$ fall into interval $i$. Breakage birth occurs in $1\sim n$. The summation of volume fractions of all intervals is unity so that $\sum_{i=1}^\infty V_i=1$.
### Discretized breakage death
The rate of breakage death is
\begin{equation}
R_i^{[2]}=S_iV_i
\end{equation}
which occurs in $2\sim n$

If it is assumed that $V_i(t)$ satisfies a discrete analogy of continuous equation,
\begin{equation}
\frac{d}{dt}V_i(t)=-S_iV_i(t)+\sum_{j=i}^\infty b_{ij}S_jV_j(t)
\end{equation}

In [250]:
def breakage(y,b,S):
# S is discretized selection function form of n vector
# b is discretized breakage function form of n*n matrix
    v = y[0:-1]
    n = len(v)
    R1 = np.zeros(n)
    R2 = np.zeros(n)
    for i in range(n):
        sum = 0
        for j in range(i,n):
            sum += b[i][j]*S[j]*v[j]
        R1[i] = sum
    # Mechanism 2 (i=2~n)
    for i in range(n):
        R2[i]=S[i]*v[i]
    dvdt = R1-R2
    dmdt = np.sum(dvdt)
    dydt = np.append(dvdt,dmdt)
    return dydt

In [62]:
a = np.array([0,1,-2])
a[0:-1]

array([0, 1])

The $k$th moment of discrete volume based breakage equation is
\begin{equation}
M_k(t)=\sum_{j=1}^\infty\bar{x}_j^k v_j(t)
\end{equation}
It is assumed that the discrete selection rate function is related to the selection rate function by
\begin{equation}
\begin{cases}
S_i=S(\bar{x}_i),\hspace{10mm}i>1\\
S_1=0
\end{cases}
\end{equation}
to prevent the particle loss from the leftmost interval $(x_0,x_1)$.

## Discretized breakage function
The rate of generation of volume of fragments from interval $i$ is
\begin{equation}
\int_{l_{i-1}}^{l_i}S(l)v(t,l)dl
\end{equation}
The discrete equivalent is $S_iV_i$.

These fragments arrive in interval $j$ at a rate
\begin{equation}
\begin{cases}
\int_{l_{i-1}}^{l_i}\int_{l_{j-1}}^{l_j}S(l)v(t,l)b(x,l)dxdl,\qquad j<i\\
\int_{l_{i-1}}^{l_i}\int_{l_{i-1}}^lS(l)v(t,l)b(x,l)dxdl,\qquad j=i
\end{cases}
\end{equation}
The discrete equivalent is $b_{ji}S_iV_i$.

Hence the discretized breakage function is
\begin{equation}
\begin{cases}
b_{ji}=\frac{\int_{l_{i-1}}^{l_i}\int_{l_{j-1}}^{l_j}S(l)v(t,l)b(x,l)dxdl}{\int_{l_{i-1}}^{l_i}S(l)v(t,l)dl}\approx\frac{\int_{l_{i-1}}^{l_i}\int_{l_{j-1}}^{l_j}S(l)b(x,l)dxdl}{\int_{l_{i-1}}^{l_i}S(l)dl},\qquad j<i\\
b_{ii}\approx\frac{\int_{l_{i-1}}^{l_i}\int_{l_{i-1}}^lS(l)b(x,l)dxdl}{\int_{l_{i-1}}^{l_i}S(l)dl}
\end{cases}
\end{equation}

For the mass conservation, the breakage function should satisfy
\begin{equation}
\sum_{j=1}^{i}b_{ji}=1
\end{equation}
\begin{align}
\sum_{j=1}^{i}b_{ji}=&\sum_{j=1}^{i-1}\frac{\int_{l_{i-1}}^{l_i}\int_{l_{j-1}}^{l_j}S(l)b(x,l)dxdl}{\int_{l_{i-1}}^{l_i}S(l)dl}+\frac{\int_{l_{i-1}}^{l_i}\int_{l_{i-1}}^lS(l)b(x,l)dxdl}{\int_{l_{i-1}}^{l_i}S(l)dl}\\
    =&\frac{\int_{l_{i-1}}^{l_i}\int_{0}^{l_{i-1}}S(l)b(x,l)dxdl}{\int_{l_{i-1}}^{l_i}S(l)dl}+\frac{\int_{l_{i-1}}^{l_i}\int_{l_{i-1}}^lS(l)b(x,l)dxdl}{\int_{l_{i-1}}^{l_i}S(l)dl}\\
    =&\frac{\int_{l_{i-1}}^{l_i}\int_{0}^{l}S(l)b(x,l)dxdl}{\int_{l_{i-1}}^{l_i}S(l)dl}\\
    =&\frac{\int_{l_{i-1}}^{l_i}S(l)\int_{0}^{l}b(x,l)dxdl}{\int_{l_{i-1}}^{l_i}S(l)dl}
\end{align}
Since $\int_{0}^{l}b(x,l)dx=1$, $\sum_{j=1}^{i}b_{ji}=1$ as desired.

## Discretized breakage function 2
The discretized breakage function from the continuous cumulative breakage function is
\begin{equation}
b_{ij}=B(l_i,l_j)-B(l_{i-1},l_j)
\end{equation}

The selection rate function can be described as (Ding, 2006)
\begin{equation}
S(l) = S_0l^p
\end{equation}
where $S_0$ is the selection rate constant. If $p=0$, the selection function is size independent.

## Volume based PBM
The volume of size $l$ is
\begin{equation}
v(t,l)=nl^3
\end{equation}
Hence,
\begin{equation}
\frac{\partial v}{\partial t}=l^3\left(\int_v^\infty S(\epsilon)b(v,\epsilon)n(\epsilon)d\epsilon-S(v)n(v)\right)
\end{equation}


In [172]:
def dsf(S0,L,p):
    # discretized selection function
    n = np.size(L)
    S = np.empty(n)
    S[0] = 0.0
    for i in range(1,n):
        S[i] = S0*L[i]**p
    return S

def vdbf(bf,Sf,L):
    # volume based discretized breakage function
    n = np.size(L)
    break_mat = np.zeros((n,n))
    def numerator(x,l):
        return Sf(l)*bf(x,l)
    def denominator(l):
        return Sf(l)      
    for i in range(n):
        if i==0:
            den,err = quad(denominator,0,L[0])
            num,err = dblquad(numerator,0,L[0],lambda x: 0,lambda x: x)
        else:
            den,err = quad(denominator,L[i-1],L[i])
            num,err = dblquad(numerator,L[i-1],L[i],lambda x: L[i-1],lambda x: x)
        assert den != 0, 'vdbf: division by zero'
        break_mat[i][i] = num/den
        for j in range(i):
            if j==0:
                num,err = dblquad(numerator,L[i-1],L[i],lambda x: 0,lambda x: L[0])
            else:
                num,err = dblquad(numerator,L[i-1],L[i],lambda x: L[j-1],lambda x: L[j])
            break_mat[j][i] = num/den
    return break_mat


In [349]:
def vdbf2(bfunc,L):
    # volume based discretized breakage function
    n = np.size(L)
    break_mat = np.zeros((n,n))
    def B(L1,L2):
        def integrand(x):
            return bfunc(x,L2)
        num,err = quad(integrand,0,L1)
        return num
    for j in range(n):
        for i in range(j+1):
            if i==0:
                break_mat[i][j] = B(L[i],L[j])
            else:
                break_mat[i][j] = B(L[i],L[j])-B(L[i-1],L[j])
    return break_mat

## Zero-th moment
Zero-th moment of volume based breakage function is total volume of particles. Therefore, time derivative of zero-th moment should be zero for the volume conservation.
\begin{align}
\frac{d}{dt}m_0&=\sum_{i=1}^n\left(-S_iV_i+\sum_{j=i}^n b_{ij}S_jV_j\right)\\
    &=-\sum_{i=1}^nS_iV_i+\sum_{i=1}^n\sum_{j=i}^nb_{ij}S_jV_j\\
    &=-\sum_{i=1}^nS_iV_i+\sum_{j=1}^n\sum_{i=1}^jb_{ij}S_jV_j\\
    &=-\sum_{i=1}^nS_iV_i+\sum_{i=1}^n\sum_{j=1}^ib_{ji}S_iV_i\\
    &=\sum_{i=1}^nS_iV_i\left(\sum_{j=1}^ib_{ji}-1\right)=0
\end{align}
Therefore, discretized breakage function satisfies volume conservation.

## Continuous mass balance equation
Let $m(l,t)dl$ be the mass of particles with sizes from $l$ to $l+dl$ at time $t$. The continuous mass balance equation is
\begin{equation}
\frac{\partial m(l,t)}{\partial t}=-S(l)m(l,t)+\int_{l}^\infty S(x)b(l,x)m(x,t)dx
\end{equation}
where $b(x,y)$ is the breakage function and $S(x)$ is the selection rate function.

The mass of particles with sizes less that $l$ is given by
\begin{equation}
M(l,t)=\int_0^lm(x,t)dx
\end{equation}
and the $k$th moment is defined as
\begin{equation}
M_k(t)=\int_0^\infty x^km(x,t)dx
\end{equation}
Integrating the mass balance equation gives
\begin{equation}
\int_0^\infty\frac{\partial m(l,t)}{\partial t}dl=\int_0^\infty\left(-S(l)m(l,t)+\int_{l}^\infty S(x)b(l,x)m(x,t)dx\right)dl
\end{equation}

So that
\begin{equation}
\frac{\text{d}M_k(t)}{\text{d}t}=
\end{equation}

The breakage function that gives uniform probability is
\begin{equation}
b(x,l)=\frac{3x^2}{l^3}
\end{equation}

The selection rate function can be described as (Ding, 2006)
\begin{equation}
S(l) = S_0l^p
\end{equation}
where $S_0$ is the selection rate constant. If $p=0$, the selection function is size independent.

In [ ]:
from scipy.special import erf

def logerf(l,lgv,sg):
    assert sg > 1, "standard deviation must be larger than 1"
    return erf(np.log(l/lgv)/(np.sqrt(2)*np.log(sg)))

def lognorm_b(x,l,lgv,sg):
    assert sg > 1, "standard deviation must be larger than 1"
    num = np.exp(-(np.log(x/lgv)/(np.sqrt(2)*np.log(sg)))**2)
    num /= (x*np.sqrt(2*np.pi)*np.log(sg))
    den = (1+logerf(l,lgv,sg))/2
    # In case 'l' is too small compared to 'lgv',
    # 'den' can be numerically zero 
    # if it is smaller than the machine precision epsilon 
    # which is not correct theoretically
    if den == 0:
        den = np.finfo(float).eps
    return num/den